# Tests for Husky PLL phase adjustment.

Checks whether the actual relative phase between target and ADC clocks is correct.

While the part 1 notebook covers related things, it doesn't care about what the phase actually **is**. This notebook does. Covers issue [501](https://github.com/newaetech/chipwhisperer/issues/501).

Some occasional failures are to be expected.

Should be run whenever changes are made to `ChipWhispererHuskyClocks.py`.

Could be a `pytest` script, however the ability to plot the clocks and visualize can be really useful when debugging.

In [ ]:
import chipwhisperer as cw

In [ ]:
scope = cw.scope()

In [ ]:
scope.default_setup()

# There are two cases to cover:
1. `EXTCLK = True`: target-supplied clock, from a CW305. Could be adapted to use a different target, but we need to be able to set its clock over a wide range.
2. `EXTCLLK = False`: no target needed; PLL uses 12 MHz XTAL reference.

In [ ]:
EXTCLK = True
#EXTCLK = False

In [ ]:
if EXTCLK:
    # setting PLL won't work without a bitfile?
    target = cw.target(scope, cw.targets.CW305, fpga_id='100t')
    target.pll.pll_enable_set(True)
    target.pll.pll_outenable_set(False, 0)
    target.pll.pll_outenable_set(True, 1)
    target.pll.pll_outenable_set(False, 2)
    target.pll.pll_outfreq_set(10e6, 1)
    MAXCLOCK = 167e6
    scope.clock.clkgen_freq = 10e6
    scope.clock.clkgen_src = 'extclk'
else:
    MAXCLOCK = 200e6
    target = None
    scope.clock.clkgen_src = 'system'
    scope.clock.clkgen_freq = 7.37e6

OVERSAMP = 20
scope.LA.enabled = True
if EXTCLK:
    scope.LA.clk_source = 'target'
else:
    scope.LA.clk_source = 'pll'
scope.LA.clkgen_enabled = False
scope.LA.oversampling_factor = OVERSAMP
scope.LA.clkgen_enabled = True
scope.LA.capture_group = 'CW 20-pin'
scope.LA.capture_depth = 200 
assert scope.LA.locked

When `EXTCLK = True`, there would be a bunch of `scope_logger.errors()` due to clock/mul settings that can't be achieved; this silences them so that the output isn't cluttered.

For the same reason, we mute warnings.

**If debugging, unset `scope.clock._quiet` to see all errors, and turn warnings back on!**

In [ ]:
cw.scope_logger.setLevel(cw.logging.ERROR)
#cw.scope_logger.setLevel(cw.logging.WARNING)

if EXTCLK:
    scope.clock._quiet = True
else:
    scope.clock._quiet = False

In [ ]:
def setup(freq, mul, phase_raw):
    global all_settings
    if EXTCLK:
        target.pll.pll_outfreq_set(freq, 1)
    scope.clock.adc_mul = mul
    scope.clock.clkgen_freq = freq
    scope.clock.adc_phase_raw = phase_raw
    assert scope.clock.pll.pll_locked
    setting = {}
    setting['freq'] = freq
    setting['mul'] = mul
    setting['phase_raw'] = phase_raw
    setting['pll_settings'] = str(scope.clock.pll)
    setting['params'] = list(scope.clock.pll.parameters)
    all_settings.append(setting)

def set_oversamp(oversamp):
    scope.LA.clkgen_enabled = False
    scope.LA.oversampling_factor = oversamp
    scope.LA.clkgen_enabled = True
    assert scope.LA.locked

def find0to1trans(data):
    pattern = [0,1]
    return [i for i in range(0,len(data)) if list(data[i:i+len(pattern)])==pattern]

def get_clocks(extclk=False, freq=None, oversamp=None):
    done = False
    count = 0
    while not done and count < 30:
        scope.LA.arm()
        scope.LA.trigger_now()
        raw = scope.LA.read_capture_data()
        adcclock = scope.LA.extract(raw, 8)
        if extclk:
            refclock = scope.LA.extract(raw, 4)
        else:
            refclock = scope.LA.extract(raw, 5)
        
        edges = find0to1trans(refclock)
        if len(edges) > 1:
            ref_edge = edges[1]
        else:
            ref_edge = edges[0]    
        
        try:
            if extclk:
                # account for fixed offset when extclk: 10ns from *falling edge* of refclock to *rising edge* of adcclock
                # we return the phase deviation from this 10ns, however we don't transform the raw clocks; they are returned as-is
                if not freq or not oversamp:
                    print('get_clocks() needs freq and oversamp when extclk is set!')
                    return 0,0,0,0
                # first, calculate the raw phase, in case it's interesting:
                adc_edges = find0to1trans(adcclock)
                adc_ref_delta2 = abs(min(adc_edges, key=lambda x:abs(x-ref_edge)) - ref_edge)
                # then calculate against the fixed 10ns offset
                edges = find0to1trans(refclock ^ 1) # get falling edge
                ref_edge = edges[1]
                # calculate 10ns in samples:
                offset = round(10e-9/(1/freq/oversamp))
                #print('offset=%d' % offset)
                #adc_edges = list(np.asarray(find0to1trans(adcclock)) - offset)
                #adc_ref_delta1 = abs(min(adc_edges, key=lambda x:abs(x-ref_edge)) - ref_edge)
                adc_ref_delta1 = find0to1trans(adcclock[ref_edge+offset:])[0]
                done = True

            else:
                adc_ref_delta1 = find0to1trans(adcclock[ref_edge:])[0]
                adc_edges = find0to1trans(adcclock)
                adc_ref_delta2 = abs(min(adc_edges, key=lambda x:abs(x-ref_edge)) - ref_edge)
                done = True
        except:
            # not sure why but sometimes the ADC clock comes back all zeros; could be an issue with the PLL or with the LA?
            # what's very strange is that this doesn't happen often, but when it does, adcclock is always all zeros, and 
            # the capture is re-attempted exactly 19 times before it's successful!
            if all(c == 0 for c in adcclock):
                adcclock = 'all zeros'
            print('could not find delta; ref_edge=%3d, lock status=%s; adcclock=%s; trying again' % (ref_edge, scope.clock.pll.pll_locked, adcclock))
            assert scope.LA.locked
            assert scope.clock.pll.pll_locked
            time.sleep(0.5)
            count += 1
            done = True # TEMP!
    return adc_ref_delta1, adc_ref_delta2, adcclock, refclock

# Phase Check

Here we pick a random clock frequency, set `adc_mul = 1`, and check that the relative phase between the clocks is ~correct for all `scope.clock.adc_phase` values.

When `EXTCLK=True`, this is difficult because the relative phase depends on the clock frequency. So we take one measurement, assume it is good, and apply an offset for all other measurements.

The last section of this notebook, "Sweep frequency with phase held at 0", can be used to explore how the phase varies with the clock.

We used to also test if the phase is monotonically increasing, but doing this *properly* is completely redundant.

In [ ]:
scope.trace.clock._warning_frequency = 303e6

In [ ]:
EXTCLK

In [ ]:
#REPS = 100
REPS = 1000

In [ ]:
from tqdm.notebook import tnrange, tqdm
import numpy as np
import time
import random

freqmuls = []
all_settings = []
la_skipped = 0
setup_skipped = 0
fails = 0
passes = 0
all_diffs = []

pbar = tqdm(total=REPS, desc='Passing')
fbar = tqdm(total=REPS, desc='Failing')
sbar = tqdm(total=REPS, desc='Skipped')

for r in tnrange(REPS):
    FREQ = random.randint(5e2, 20e2)*1e4 # lower resolution gets much faster setting of the CW305 PLL
    OVERSAMP = int(300e6//FREQ)
    samples = OVERSAMP*2
    if EXTCLK:
        # in this case, the ADC clock and target-generated clock can both be shifting around relative to each other, so we need to tolerate a larger deviance:
        mintol = 3
    else:
        mintol = 2
    TOL = max(mintol, OVERSAMP//30)
    MUL = 1
    freqmuls.append([FREQ, MUL])
    try:
        setup(FREQ, MUL, 0)
        set_oversamp(OVERSAMP)
        time.sleep(0.5)
        if abs(scope.LA.sampling_clock_frequency/FREQ - OVERSAMP) / OVERSAMP * 100 > 1:
            la_skipped += 1
            sbar.update(1)
            continue
    except Exception as e:
        print('failed to setup: %s' % e)
        setup_skipped += 1
        continue

    # Now sweep over all raw phase steps, but don't go over one full clock period for speed and simplicity
    # (this way we (should) end up with a strictly monotonically increasing relative phase).
    # Note that often we cannot cover a full clock period!
    if EXTCLK:
        start_phase = 0
        steps = min(32, scope.clock.pll.get_outdiv(3)) # 32 is the max range of phase adjustments (0 to +31)
    else:
        start_phase = -min(31, scope.clock.pll.get_outdiv(3))
        steps = min(63, scope.clock.pll.get_outdiv(3)) # 63 is the max range of phase adjustments (-31 to +31)


    measured_phases = []
    adcs = []
    refs = []

    for phase_raw in range(start_phase, start_phase+steps):
        scope.clock.adc_phase_raw = phase_raw
        measured_phase, phase2, adc, ref = get_clocks(EXTCLK, FREQ, OVERSAMP)
        settings = str(scope.clock.pll)
        params = scope.clock.pll.parameters
        sampling_clock = scope.LA.sampling_clock_frequency

        offset = find0to1trans(ref)[0]
        adcs.append(adc[offset:offset+samples])
        refs.append(ref[offset:offset+samples])
        measured_phases.append(measured_phase)

        # compare measured phase against expected:
        expected_measurement = OVERSAMP * scope.clock.adc_phase/100
        # could be off by a full period:
        diff = expected_measurement - measured_phase
        diffs = [abs(diff), abs(diff+OVERSAMP), abs(diff-OVERSAMP)]
        all_diffs.append(min(diffs))
        if min(diffs) > TOL:
            print('FAIL! freq=%d, oversamp=%d, phase_raw=%d, diffs=%s' % (FREQ, OVERSAMP, phase_raw, diffs))
            fails += 1
            fbar.update(1)
        else:
            passes += 1
            pbar.update(1)


In [ ]:
assert fails == 0

In [ ]:
assert passes/(la_skipped+setup_skipped) > 4

In [ ]:
la_skipped, setup_skipped

**This concludes the test; the rest of this notebook is to help diagnose failures.**

## replicate a failure:

In [ ]:
#FREQ = freqmuls[0][0]
#FREQ = 5e6
#FREQ = 10e6
FREQ = 20e6
#FREQ = 5130000

MUL = 1

setup(FREQ, MUL, 0)
OVERSAMP = int(300e6//FREQ)
set_oversamp(OVERSAMP)
samples = OVERSAMP*2

scope.clock.adc_phase_raw = 0
measured_phase, phase2, adc, ref = get_clocks(EXTCLK, FREQ, OVERSAMP)

In [ ]:
expected_measurement = OVERSAMP * scope.clock.adc_phase/100
# could be off by a full period:
diff = expected_measurement-measured_phase
diffs = [abs(diff), abs(diff+OVERSAMP)]
all_diffs.append(min(diffs))
if min(diffs) > TOL:
    print('FAIL! freq=%d, oversamp=%d, phase_raw=%d, diff=%d' % (FREQ, OVERSAMP, phase_raw, min(diffs)))

In [ ]:
if EXTCLK:
    start_phase = 0
    steps = min(32, scope.clock.pll.get_outdiv(3)) # 32 is the max range of phase adjustments (0 to +31)
else:
    start_phase = -min(31, scope.clock.pll.get_outdiv(3))
    steps = min(63, scope.clock.pll.get_outdiv(3)) # 63 is the max range of phase adjustments (-31 to +31)

measured_phases = []
measured_phases2 = []
adcs = []
refs = []

for phase_raw in tnrange(start_phase, start_phase+steps):
    scope.clock.adc_phase_raw = phase_raw
    measured_phase, phase2, adc, ref = get_clocks(EXTCLK, FREQ, OVERSAMP)
    settings = str(scope.clock.pll)
    params = scope.clock.pll.parameters
    sampling_clock = scope.LA.sampling_clock_frequency

    offset = find0to1trans(ref)[0]
    adcs.append(adc[offset:offset+samples])
    refs.append(ref[offset:offset+samples])
    measured_phases.append(measured_phase)
    measured_phases2.append(phase2)

    # compare measured phase against expected:
    expected_measurement = OVERSAMP * scope.clock.adc_phase/100
    # could be off by a full period:
    diff = expected_measurement - measured_phase
    diffs = [abs(diff), abs(diff+OVERSAMP), abs(diff-OVERSAMP)]
    all_diffs.append(min(diffs))
    if min(diffs) > TOL:
        print('FAIL! freq=%d, oversamp=%d, phase_raw=%d, diffs=%s' % (FREQ, OVERSAMP, phase_raw, diffs))


## visualize phase monotonicity:

In [ ]:
from bokeh.plotting import figure, show
from bokeh.resources import INLINE
from bokeh.io import output_notebook

output_notebook(INLINE)

In [ ]:
# test if monotonically increasing (with a shift allowed)
# won't work if we capture more than one period!
reordered_phases = []
reorder = None
for i in range(1,len(measured_phases)):
    #if measured_phases[i] < measured_phases[i-1]:
    if measured_phases[i] < measured_phases[i-1] - OVERSAMP//2:
        reorder = i
if reorder:
    reordered_phases = measured_phases[reorder:] +  measured_phases[:reorder]
else:
    reordered_phases = measured_phases

TOL = 1
for i in range(1,len(reordered_phases)):
    if reordered_phases[i] + TOL < reordered_phases[i-1]:
        print('FAIL! i=%d' % i)


In [ ]:
xrange = list(range(len(measured_phases)))
p = figure(width=1800)
p.line(xrange, measured_phases, line_color='green')
#p.line(xrange, measured_phases2, line_color='red')
p.line(xrange, reordered_phases, line_color='blue')
show(p)

In [ ]:
def update_plot(p):
    S1.data_source.data['y'] = refs[p][:samples] + 2
    S2.data_source.data['y'] = adcs[p][:samples] + 0  
    push_notebook()

In [ ]:
from ipywidgets import interact, Layout
from bokeh.io import push_notebook
from bokeh.models import Span, Legend, LegendItem

p = 0

S = figure(width=1800)

samples = int(OVERSAMP*2)
xrange = list(range(samples))

S1 = S.line(xrange, refs[p][:samples] + 2, line_color='black')
S2 = S.line(xrange, adcs[p][:samples] + 0, line_color='blue')

In [ ]:
show(S, notebook_handle=True)

In [ ]:
interact(update_plot, p=(0, len(adcs)-1))

## Sweep frequency with phase held at 0:

In [ ]:
scope.LA.capture_depth = 500

In [ ]:
samples = 300e6//5e6*4

In [ ]:
scope.userio.mode = 'fpga_debug'
scope.userio.fpga_mode = 1

In [ ]:
all_settings = []

In [ ]:
from tqdm.notebook import tnrange, tqdm
import numpy as np
import time
import random

measured_phases = []
measured_phases2 = []
adcs = []
refs = []
all_settings = []
samples = None

for FREQ in tnrange(int(5e6), int(30e6), int(2e5)):
    MUL = 1
    setup(FREQ, MUL, 0)
    OVERSAMP = int(300e6//FREQ)
    set_oversamp(OVERSAMP)
    if samples is None:
        samples = OVERSAMP*4
    
    scope.clock.adc_phase_raw = 0
    measured_phase, phase2, adc, ref = get_clocks(EXTCLK, FREQ, OVERSAMP)

    offset = find0to1trans(ref)[0]
    adcs.append(adc[offset:offset+samples])
    refs.append(ref[offset:offset+samples])
    measured_phases.append(measured_phase)
    measured_phases2.append(phase2)


In [ ]:
from bokeh.plotting import figure, show
from bokeh.resources import INLINE
from bokeh.io import output_notebook

output_notebook(INLINE)

In [ ]:
xrange = list(range(len(measured_phases)))
p = figure(width=1800)
p.line(xrange, measured_phases, line_color='green')
#p.line(xrange, measured_phases2, line_color='blue')
show(p)

In [ ]:
def update_plot(f):
    S1.data_source.data['y'] = refs[f][:samples] + 2
    S2.data_source.data['y'] = adcs[f][:samples] + 0  
    push_notebook()

In [ ]:
from ipywidgets import interact, Layout
from bokeh.io import push_notebook
from bokeh.models import Span, Legend, LegendItem

p = 0

S = figure(width=1800)

samples = 80
xrange = list(range(samples))

S1 = S.line(xrange, refs[p][:samples] + 2, line_color='black')
S2 = S.line(xrange, adcs[p][:samples] + 0, line_color='blue')

In [ ]:
show(S, notebook_handle=True)

In [ ]:
interact(update_plot, f=(0, len(adcs)-1))